In [71]:
import pandas as pd

In [152]:
df = pd.read_csv('chat_data.csv', usecols=[0,1])
df = df[:-2]
df.head()

,Q,A
0,침구 추가를 해야 할 것 같아서요,1세트가 필요하신 건가요
1,맞습니다 추가 요금은 얼마인가요,1만 원이시고요 투숙기간 과는 무관합니다
2,침구문의는 어디다 말씀드려야 할까요,프론트에다가 사전에 말씀 부탁 드릴게요
3,예약하려면 어떻게 해야 하나요,회원 가입 후 로그인 하셔서 예약하시면 되세요
4,객실엔 세면 도구가 있나요,타월과 욕실 용품은 있지만 개인 용품은 챙겨오셔야 합니다


In [153]:
# 결측치 확인
df.isnull().sum()

Q    0
A    0
dtype: int64

In [154]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [155]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

In [156]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

51200
51200
51200
----------
</s>
<usr>
<pad>
<sys>


In [157]:
import tqdm     # 반복문 진행률

In [158]:
len(df)

34336

In [159]:
batch_size = 32

In [160]:
def get_chat_data():
    for question, answer in zip(df.Q.to_list(), df.A.to_list()):
        # start of sentence
        bos_token = [tokenizer.bos_token_id]

        # end of sentence
        eos_token = [tokenizer.eos_token_id]

        sent = tokenizer.encode('' + question + '' + answer)

        # 결과값을 나누어 얻음
        yield bos_token + sent + eos_token

In [161]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types = tf.int32)

In [162]:
dataset = dataset.padded_batch(batch_size = batch_size, padded_shapes=(None,),
padding_values=tokenizer.pad_token_id)

In [163]:
for batch in dataset:
    print(batch.shape)

(32, 36)
(32, 31)
(32, 26)
(32, 49)
(32, 50)
(32, 51)
(32, 44)
(32, 55)
(32, 57)
(32, 62)
(32, 50)
(32, 59)
(32, 51)
(32, 53)
(32, 49)
(32, 58)
(32, 54)
(32, 45)
(32, 49)
(32, 45)
(32, 50)
(32, 52)
(32, 64)
(32, 52)
(32, 52)
(32, 49)
(32, 42)
(32, 44)
(32, 50)
(32, 41)
(32, 40)
(32, 45)
(32, 40)
(32, 37)
(32, 45)
(32, 52)
(32, 55)
(32, 46)
(32, 41)
(32, 57)
(32, 39)
(32, 41)
(32, 39)
(32, 50)
(32, 52)
(32, 46)
(32, 50)
(32, 43)
(32, 53)
(32, 48)
(32, 51)
(32, 47)
(32, 57)
(32, 43)
(32, 51)
(32, 48)
(32, 52)
(32, 47)
(32, 48)
(32, 53)
(32, 47)
(32, 41)
(32, 59)
(32, 54)
(32, 58)
(32, 50)
(32, 48)
(32, 74)
(32, 65)
(32, 35)
(32, 56)
(32, 56)
(32, 44)
(32, 48)
(32, 50)
(32, 56)
(32, 41)
(32, 40)
(32, 35)
(32, 39)
(32, 48)
(32, 50)
(32, 53)
(32, 48)
(32, 52)
(32, 55)
(32, 46)
(32, 47)
(32, 51)
(32, 47)
(32, 52)
(32, 41)
(32, 42)
(32, 39)
(32, 35)
(32, 26)
(32, 25)
(32, 30)
(32, 33)
(32, 32)
(32, 46)
(32, 54)
(32, 46)
(32, 55)
(32, 50)
(32, 51)
(32, 60)
(32, 54)
(32, 58)
(32, 54)
(32, 38)
(

In [164]:
tokenizer.decode(batch[0])

'<|endoftext|> 제가 지금 해운대에서 여행하면서 공연을 보려고 예매했는데요 사이트에서 안내된 거랑 다르게 추가로 결제가 더 된 것 같아서요 고객님 영화의 전당 사이트에서 예매하셨을까요 <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [165]:
print(tokenizer.encode('네 예약하려면 어떻게 해야 하나요'))

[9873, 34791, 19352, 11649, 13023, 9402, 8084]


In [166]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [167]:
steps = len(df) // batch_size + 1
print(steps)

1074


In [168]:
batch.shape

TensorShape([32, 54])

In [169]:
EPOCHS = 3

for epoch in range(EPOCHS):
    epoch_loss = 0

    for batch in tqdm.notebook.tqdm(dataset, total=steps):
        # tf.GradientTape() : 실행된 모든 연산은 테이프에 기록
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)
            loss = result[0]
            batch_loss = tf.reduce_mean(loss)
        
        grads = tape.gradient(batch_loss, model.trainable_variables)
        adam.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss += batch_loss / steps

    print('[Epoch: {:>4}] cost = {:>9}'.format(epoch + 1, epoch_loss))

  0%|          | 0/1074 [00:00<?, ?it/s]

InvalidArgumentError: Exception encountered when calling layer "transformer" (type TFGPT2MainLayer).

input_ids must be smaller than the embedding layer's input dimension (got 51200 >= 51200)
Condition x < y did not hold.
First 3 elements of x:
[51200  9665  6919]
First 1 elements of y:
[51200]

Call arguments received by layer "transformer" (type TFGPT2MainLayer):
  • input_ids=tf.Tensor(shape=(32, 36), dtype=int32)
  • past_key_values=None
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • use_cache=True
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False